In [4]:
example = torch.arange(3*4*4).reshape(3, 4, 4)
example

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15]],

        [[16, 17, 18, 19],
         [20, 21, 22, 23],
         [24, 25, 26, 27],
         [28, 29, 30, 31]],

        [[32, 33, 34, 35],
         [36, 37, 38, 39],
         [40, 41, 42, 43],
         [44, 45, 46, 47]]])

In [45]:
LP = LinearProjector(4, 18)

In [42]:
def break_into_patches(input, N):
    """break tensor input image into N patches"""
    H, W = tuple(input.shape[-2:])
    P = int(np.sqrt((H*W)/N))

    split1 = torch.split(input, P, -1)
    split2 = []
    for split in split1:
        split2.extend(list(torch.split(split, P, -2)))
    return torch.stack(split2)    

In [5]:
import numpy as np
input_size = 64
N = 16
P = int(np.sqrt((input_size**2)/N))
a = torch.arange(2*(input_size**2)*3).reshape(2, 3, input_size, input_size)
split1 = torch.split(a, P, -1)

split2 = []
for split in split1:
    split2.extend(list(torch.split(split, P, -2)))

# split2 = (torch.split(splitn, 2, 1) for splitn in split1)
# split2
print(a.shape)
torch.stack(split2).shape

16
torch.Size([2, 3, 64, 64])


torch.Size([16, 2, 3, 16, 16])

In [52]:
from PIL import Image
from torchvision import transforms
img = Image.open('0005098900241653513286999.jpg').convert('RGB')
img = np.stack([np.array(img)]*10)
imgt = torch.from_numpy(img).permute(0, 3, 1, 2)
imgt = imgt.type(torch.float64)

t = transforms.RandomCrop(464, pad_if_needed=True)
imgt = t(imgt)


In [66]:
import torchvision
c = break_into_patches(imgt, 16)
print(c.shape)
# torchvision.utils.make_grid(c[0])

torch.Size([16, 10, 3, 116, 116])


In [72]:
d = c.permute(1, 0, 2, 3, 4)

In [78]:
d = d.flatten(-3, -1)

torch.Size([10, 16, 40368])

In [103]:
a = torch.arange(16).repeat(10, 1).unsqueeze(2)
a.shape

torch.Size([10, 16, 1])

In [114]:
e = torch.cat((a, d), 2)
e = e.to(torch.float32)

In [107]:
from torch import nn

In [120]:
P = 116
N = 10

In [123]:
e.shape

torch.Size([10, 16, 40369])

In [126]:
ls = nn.ModuleList([
    nn.Linear((P**2)*3 +1, 5) for _ in range(N)
])

torch.cat([ls[i](e[:, i]) for i in range(N)]).shape

torch.Size([100, 5])

In [127]:
e[:, 0].shape

torch.Size([10, 40369])

In [49]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

plt.rcParams["savefig.bbox"] = 'tight'

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

OK LETS TRY IT

In [1]:
cd ..

/Users/adil.quettawala/Documents/APS360/Project/APS360-project


In [2]:
from transformer_images import *
import torch

In [3]:
LP = LinearProjector(16, 5)

from PIL import Image
from torchvision import transforms

img = Image.open('0005098900241653513286999.jpg').convert('RGB')
img = np.stack([np.array(img)]*10)
imgt = torch.from_numpy(img).permute(0, 3, 1, 2)
imgt = imgt.type(torch.float64)

t = transforms.RandomCrop(464, pad_if_needed=True)
imgt = t(imgt)

In [4]:
imgt.shape

torch.Size([10, 3, 464, 464])

In [6]:
out = LP.forward(imgt)

In [7]:
out.shape

torch.Size([10, 16, 6])

In [11]:
embed = torch.arange(1, 17).repeat(10, 1).unsqueeze(-1)
embed.shape

torch.Size([10, 16, 1])

In [15]:
out_embed = torch.cat((embed, out), -1)

tensor([[[   1.0000,  305.6849,  209.2019, -106.8989,  243.2090,   83.9843],
         [   2.0000,   28.4007,   13.6438,  135.1077,  -97.4835, -166.9490],
         [   3.0000,  157.5540,   74.9798,  -74.5945,    4.2159,  -64.9919],
         [   4.0000,  -52.3303, -163.3389,  270.7256,  228.7937,    1.1391],
         [   5.0000,  -58.9649,   30.6662, -136.0254,  137.2398,  -67.5760],
         [   6.0000,  127.0598, -139.3164,  -98.9900,  -46.4051,  151.0910],
         [   7.0000,  278.4673, -235.2512,  -48.6796,    2.8498,  -88.9378],
         [   8.0000, -223.0407,  -79.0655, -167.6257, -130.0516,  114.7287],
         [   9.0000,  -93.0002,  -20.9926,   72.6275,  -28.3302,   95.6224],
         [  10.0000,   98.8691, -213.1415,   71.3390,  138.2855,  248.8171],
         [  11.0000, -217.5755, -174.6820,  -66.4536,   67.2535,  -16.0954],
         [  12.0000,   -8.3566,  -40.6561,   71.3283, -160.6714,  116.9346],
         [  13.0000, -338.9627, -223.6482,  -61.3419,   73.0948,  156.0408],

In [15]:
class_embed = (
            torch.randn(6)
            .unsqueeze(0)
        )
class_embed.shape

torch.Size([1, 6])

In [16]:
class_embed[:, 0] = 0.0
ce = class_embed.repeat(10, 1).unsqueeze(1)

In [17]:
out.shape

torch.Size([10, 16, 6])

In [18]:
ce.shape

torch.Size([10, 1, 6])

In [20]:
torch.cat((ce, out), 1).shape

torch.Size([10, 17, 6])

In [35]:
nn.Parameter(class_embed)

Parameter containing:
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]], requires_grad=True)

In [26]:
out1 = torch.cat(out, 1)
out2 = torch.cat(out, 1).reshape(10, 16, 5)

In [10]:
LP._fc_list[0]._parameters['weight'].shape

torch.Size([5, 40368])

ONE MORE TIME

In [1]:
cd ..

/Users/adil.quettawala/Documents/APS360/Project/APS360-project


In [2]:
from transformer_images import *
import torch

In [3]:
LP = LinearProjector(16, 5)

from PIL import Image
from torchvision import transforms

img = Image.open('0005098900241653513286999.jpg').convert('RGB')
img = np.stack([np.array(img)]*10)
imgt = torch.from_numpy(img).permute(0, 3, 1, 2)
imgt = imgt.type(torch.float64)

t = transforms.RandomCrop(464, pad_if_needed=True)
imgt = t(imgt)

In [4]:
imgt.shape

torch.Size([10, 3, 464, 464])

In [5]:
out = LP.forward(imgt)

In [6]:
out.shape

torch.Size([10, 17, 6])

In [1]:
cd ..

/Users/adil.quettawala/Documents/APS360/Project/APS360-project


In [2]:
from transformer_images import *

IT = ImageTransformer('ViT', 10, 0.01, 200, 16, 5, 1, 1)
IT

/Users/adil.quettawala/.virtualenvs/global/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


ImageTransformer(
  (criterion): MSELoss()
  (_LP): LinearProjector(
    (_fc_list): ModuleList(
      (0-16): 17 x Linear(in_features=40368, out_features=4, bias=True)
    )
  )
  (_LN): LayerNorm((17, 5), eps=1e-05, elementwise_affine=True)
  (_TEL): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=5, out_features=5, bias=True)
    )
    (linear1): Linear(in_features=5, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=5, bias=True)
    (norm1): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (_TE): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLin

In [4]:
IT(imgt)

tensor([[0.3911],
        [0.3177],
        [0.2505],
        [0.2337],
        [0.3913],
        [0.3362],
        [0.3125],
        [0.3564],
        [0.2784],
        [0.3462]], grad_fn=<AddmmBackward0>)

In [1]:
cd ..

/home/compute/APS-360


In [2]:
import torch
import torch.nn as nn
import pandas as pd
from common import model
from common import evaluate
from torch.utils.data import Dataset, DataLoader

from transformer_images import *

In [3]:
# VIT = ImageTransformer('VITb', b, 0.01, 100, 16, 10, 1, 5)
train, val = get_data_loaders(VIT.batch_size)
# model.train_model(VIT, train, val)

In [10]:
VIT = ImageTransformer('VITb', 1, 0.01, 100, 16, 10, 1, 5)
VIT.to('cuda')

ImageTransformer(
  (criterion): MSELoss()
  (_LP): LinearProjector(
    (_fc_list): ModuleList(
      (0-16): 17 x Linear(in_features=40368, out_features=9, bias=True)
    )
  )
  (_LN): LayerNorm((17, 10), eps=1e-05, elementwise_affine=True)
  (_TEL): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=10, out_features=10, bias=True)
    )
    (linear1): Linear(in_features=10, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=10, bias=True)
    (norm1): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (_TE): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantiz

In [8]:
for t in train:
    print(t[0].device)
    print(t[1].device)
    break

for v in val:
    print(v[0].device)
    print(v[1].device)
    break

cuda:0
cuda:0
cuda:0
cuda:0
